In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
df.info()

In [ ]:
df[["title", "author", "keywords", "commentCount"]]

In [ ]:
df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

In [ ]:
from collections import Counter
keywords = Counter([keyword for keywords in df["keywords"] for keyword in str(keywords).split(", ")])

In [ ]:
keywords.most_common(20)

In [ ]:
df["commentCount"].dropna().map(int).plot.hist(bins=40)

In [ ]:
df["normalizedCommentCount"] = df["commentCount"].fillna(0).map(int)
df.loc[df["normalizedCommentCount"]>500, "normalizedCommentCount"] = 500

In [ ]:
df["normalizedCommentCount"].plot.hist(bins=40)

In [ ]:
print(len(df[df["normalizedCommentCount"]<10]))
print(len(df[df["normalizedCommentCount"]>50]))